In [4]:
from joshpyutil import mpl
import numpy as np
import scipy.interpolate
import tqdm


n = 2000
N = 100000
terrain_r = 0.2
terrain_h = 0.3
gamma = 2
R = 0.00001
g = 1000

x, delta_x = np.linspace(0, 1, n, retstep=True)


def grad(field):
    spline = scipy.interpolate.InterpolatedUnivariateSpline(x, field)
    return spline.derivative()(x)


terrain = (
    terrain_h * np.exp(-(x - 1/2)**2 / terrain_r**2)
    + 0.4 * terrain_h * np.sin(14 * np.pi * (x - 1/2) + 1)
)
grad_terrain = grad(terrain)
water = np.zeros(n)
moment = np.zeros(n)

t = 0
with mpl.autovideo('video.mp4', 3, frame_rate_hz=10, size_inches=(6, 6), sharex=True) as av:
    for i in tqdm.tqdm(range(N)):
        dt = 0.00001
        t += dt

        grad_water = grad(water)
        accel_grav = -g / (1 + (grad_water + grad_terrain)**2) * (grad_water + grad_terrain)

        water_flux = moment.copy()
        velocity = moment.copy()
        tiny_mask = water < 0.001
        velocity[tiny_mask] = 0
        velocity[~tiny_mask] /= water[~tiny_mask]
        moment_flux = moment * velocity

        # Lax method.
        new_water = np.zeros_like(water)
        new_water[1:-1] = 1/2 * (water[:-2] + water[2:])
        new_water[1:-1] -= dt/2 * 1/delta_x * (water_flux[2:] - water_flux[:-2])
        new_water[0] = new_water[1]
        new_water[-1] = new_water[-2]
        if True:
            new_water[3 * n // 7:4 * n // 7] += R
        new_water[new_water < 0] = 0
        water[:] = new_water

        # Lax method.
        new_moment = np.zeros(n)
        new_moment[1:-1] = 1/2 * (moment[:-2] + moment[2:])
        new_moment[1:-1] -= dt/2 * 1/delta_x * (moment_flux[2:] - moment_flux[:-2])
        water_cutoff = 0.01
        new_moment += dt * (water * (water < water_cutoff) + water_cutoff * (water >= water_cutoff)) * accel_grav
        new_moment -= dt * gamma * moment[:]
        new_moment[0] = new_moment[1]
        new_moment[-1] = new_moment[-2]
        if new_moment[0] > 0:
            new_moment[0] = 0
        if new_moment[-1] < 0:
            new_moment[-1] = 0
        moment[:] = new_moment

        if i % (N // 100) == 0:
            with av.next_frame() as ap:
                min_terrain = terrain.min()
                _ = ap.ax.fill_between(x, min_terrain, terrain, label='terrain', color='sienna')
                _ = ap.ax.fill_between(x, terrain, terrain + water, label='water', color='lightblue')
                ap.set(xlim=[0, 1])

                ap = ap.next()
                ap.plot(x, moment, label='p')
                ap.legend()

                ap = ap.next()
                # ap.plot(
                #     x[1:-1],
                #     (water[1:-1] * accel_grav[1:-1])*dt
                #     - (1/2 * 1/delta_x * (moment_flux[2:] - moment_flux[:-2]))*dt,
                #     label=r'$dp/dt \Delta t$')
                ap.plot(
                    x,
                    new_moment - moment,
                    label=r'$dp/dt \Delta t$')
                ap.legend()

print('t:', t)
print('water.max():', water.max())

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [01:06<00:00, 1511.81it/s]


t: 0.9999999999980838
water.max(): 0.2369031508335932
